In [14]:
from importlib import reload
import sys
from imp import reload
import warnings
warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

In [15]:
import pandas as pd

df = pd.read_json('datasets/news.json', lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [16]:
df = df.drop(['authors', 'link', 'date'], axis=1)
df.head()

,category,headline,short_description
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ..."


In [17]:
df['information'] = df[['headline', 'short_description']].apply(lambda x: ' '.join(x), axis=1)
df.head()

,category,headline,short_description,information
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...


In [18]:
news = df
news.to_csv(r'datasets/news.csv', index=False)

In [19]:
df = df.drop(['headline', 'short_description'], axis=1)
df.head()

,category,information
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...


In [20]:
categories = df.groupby('category').size().sort_values(ascending=False)
categories

category
POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL             9887
STYLE & BEAUTY     9649
PARENTING          8677
HEALTHY LIVING     6694
QUEER VOICES       6314
FOOD & DRINK       6226
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3651
WOMEN              3490
IMPACT             3459
DIVORCE            3426
CRIME              3405
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
WORLDPOST          2579
RELIGION           2556
STYLE              2254
SCIENCE            2178
WORLD NEWS         2177
TASTE              2096
TECH               2082
MONEY              1707
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
ARTS & CULTURE     1339
ENVIRONMENT        1323
COLLEGE            1144
LATINO VOICES      1129
CULTURE & ARTS     1030
EDUCATION          1004
dtype: 

In [21]:
from sklearn.model_selection import train_test_split

TOP_N_CATEGORIES = 15

df = df[df.category.apply(lambda x: x in categories.index[:TOP_N_CATEGORIES]) &\
            (df.information.apply(len) > 0)]
df_train, df_test = train_test_split(df, test_size=.1, random_state=31)

In [22]:
df_train.head()

,category,information
126261,TRAVEL,10 Reasons You Should ALWAYS Ask For The Windo...
105073,ENTERTAINMENT,Nicki Minaj's New Song Reveals She Almost Had ...
22595,POLITICS,GOP Donor Who Reportedly Sought Clinton Emails...
78861,ENTERTAINMENT,Aaron Sorkin Slams Sony Hack As 'Terrorism' Th...
50575,BLACK VOICES,Bishop Eddie Long On Weight Loss: I Stopped Ea...


In [23]:
MAX_SEQUENCE_LEN = 25
UNK = 'UNK'
PAD = 'PAD'

def text_to_id_list(text, dictionary):
    return [dictionary.token2id.get(tok, dictionary.token2id.get(UNK))
            for tok in text_to_tokens(text)]

def texts_to_input(texts, dictionary):
    return sequence.pad_sequences(
        list(map(lambda x: text_to_id_list(x, dictionary), texts)), maxlen=MAX_SEQUENCE_LEN,
        padding='post', truncating='post', value=dictionary.token2id.get(PAD))

def text_to_tokens(text):
    return [tok.text.lower() for tok in nlp.tokenizer(text)
            if not (tok.is_punct or tok.is_quote)]

def build_dictionary(texts):
    d = Dictionary(text_to_tokens(t)for t in texts)
    d.filter_extremes(no_below=3, no_above=1)
    d.add_documents([[UNK, PAD]])
    d.compactify()
    return d

In [36]:
from gensim.corpora import Dictionary
import spacy
nlp = spacy.load('en')

dictionary = build_dictionary(df.information)

In [25]:
from tensorflow.keras.preprocessing import sequence

x_train = texts_to_input(df_train.information, dictionary)
x_test = texts_to_input(df_test.information, dictionary)

In [37]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(categories.index[:TOP_N_CATEGORIES])
y_train = lb.transform(df_train.category)
y_test = lb.transform(df_test.category)

In [38]:
x_train[:5]

array([[ 2373,  1516,   145,   203,  1522,   658,     7,    17,  8440,
         1773,   392,   594,   145,  1959,   598,   128,    17,  8440,
         1773,     2,  2255,    17,    19,    64,   224],
       [ 4901,  4900,     0,    54,    16,   135,   895,  1119,   170,
            2,  1897, 35387, 35387, 35387, 35387, 35387, 35387, 35387,
        35387, 35387, 35387, 35387, 35387, 35387, 35387],
       [ 1389,  7330,   188,   656,  1627,   660,   664,  5683,  1052,
         1141,   660, 11628,    32,  4419,  8283,  9157,    15,   656,
         1627,    73, 10606, 18738,   660,   664,     3],
       [ 2183, 19508,  2617, 12909,  5640,   272,  7942,    17,  9163,
          575,    17,  1717,  1593,    40,  9624,  8327,    73,  7942,
          392, 19556, 18738,   664, 35387, 35387, 35387],
       [  369,  8122,   488,   111,  7054,  3894,   802,  3903,  6567,
          168,     2, 10227, 13884,  7232, 13154,     3, 17788,  6264,
        35388,   938,    30, 10514,  3013,    73,    17]],

In [39]:
dictionary.save('dictionary')

In [40]:
loaded_dict = Dictionary.load('dictionary')

In [41]:
len(loaded_dict)

35389

In [44]:
y_train[:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [46]:
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

import numpy as np

max_features = 1000
# tokenizer = Tokenizer(num_words=max_features)
# tokenizer.fit_on_texts(x_train)
# list_tokenized_train = tokenizer.texts_to_sequences(x_train)

maxlen = 25
# X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)

embed_size = 18
model = Sequential()
model.add(Embedding(len(dictionary), embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(15, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 102776 samples, validate on 25694 samples
Epoch 1/3
102776/102776 [==============================] - 225s 2ms/sample - loss: 0.2230 - acc: 0.9351 - val_loss: 0.1655 - val_acc: 0.9441
Epoch 2/3
102776/102776 [==============================] - 231s 2ms/sample - loss: 0.1430 - acc: 0.9510 - val_loss: 0.1237 - val_acc: 0.9576
Epoch 3/3
102776/102776 [==============================] - 221s 2ms/sample - loss: 0.1083 - acc: 0.9624 - val_loss: 0.1126 - val_acc: 0.9615


In [47]:
import numpy as np
from sklearn.metrics import classification_report

y_test_pred = [lb.classes_[i] for i in np.argmax(model.predict(x_test), axis=1)]
print(classification_report(df_test.category, y_test_pred))

                precision    recall  f1-score   support

  BLACK VOICES       0.23      0.02      0.03       447
      BUSINESS       0.61      0.25      0.35       631
        COMEDY       0.67      0.05      0.10       492
 ENTERTAINMENT       0.56      0.80      0.66      1543
  FOOD & DRINK       0.70      0.73      0.72       595
HEALTHY LIVING       0.54      0.35      0.42       706
 HOME & LIVING       0.60      0.38      0.47       412
     PARENTING       0.58      0.75      0.65       872
       PARENTS       0.45      0.26      0.33       368
      POLITICS       0.82      0.91      0.86      3254
  QUEER VOICES       0.69      0.69      0.69       661
        SPORTS       0.49      0.47      0.48       489
STYLE & BEAUTY       0.75      0.82      0.79       984
        TRAVEL       0.71      0.77      0.74      1034
      WELLNESS       0.68      0.82      0.74      1787

     micro avg       0.68      0.68      0.68     14275
     macro avg       0.61      0.54      0.54 

In [54]:
x_train[22:23]

array([[ 2692, 13583, 28448,   589,   111,  2436,   117,  1352,   647,
         1489,   145,   531,    17, 11823,    13,  2692, 13583, 28448,
        35388,   845, 14823,   111,  4040,   731,     0]], dtype=int32)

In [55]:
y_train[22:23]

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [56]:
x =  model.predict(x_train[22:23])
x

array([[0.23557761, 0.01862416, 0.07511178, 0.13568836, 0.00231984,
        0.00206062, 0.02323294, 0.02467659, 0.00796232, 0.03074744,
        0.09912938, 0.261585  , 0.06513786, 0.13051404, 0.00172663]],
      dtype=float32)

In [57]:
model.save('models/news/news_model.h5')

In [58]:
rev1 = lb.inverse_transform(y_train)
rev2 = lb.inverse_transform(y_test)

In [87]:
rev1[2213]

'HOME & LIVING'

In [88]:
y_train[2213]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
[[ black voices, business, comedy, entertainment, foodDrink, healthy living, homeLiving, parenting, parents, politics, queer voices, sports, styleBeauty, travel, wellness]]